In [1]:
import time
import torch
import json
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math

from tqdm import tqdm
from PIL import Image
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Subset, Dataset, DataLoader
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck, ResNet18_Weights, ResNet34_Weights, resnet18
from torchvision.datasets import ImageFolder
from utils.loss_functions import tkd_kdloss

from models_package.models import Teacher, Student
import models_package
from utils.compare_tools import compare_model_size, compare_inference_time, compare_performance_metrics, plot_comparison
from utils.misc_tools import best_LR, best_LR_wider, train_teacher, retrieve_teacher_class_weights, new_teacher_class_weights

In [2]:
# Hyperparameters
learning_rate = 0.001379 # 0.096779
num_epochs = 2 # 200
temperature = 4.0
alpha = 0.9
momentum = 0.9
num_classes = 30
step_size = 30
gamma = 0.1
patience = 7  # for early stopping
lmda = 3
batch_size = 256
num_workers = 4

class_labels = [0, 1, 3, 4, 6, 7, 11, 15, 17, 18, 19, 20, 22, 25, 27, 28, 30, 31, 33, 35, 36, 37, 39, 43, 44, 50, 51, 54, 57, 58]
class_labels_new = torch.tensor([i for i in range(len(class_labels))])



In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
len(class_labels)

30

## Test dataloader from utils

In [5]:
from data.data_loader import load_wider

train_file = ['data/wider/trainval_wider.json']
test_file = ['data/wider/test_wider.json']

class_labels = [0, 1, 3, 4, 6, 7, 11, 15, 17, 18, 19, 20, 22, 25, 27, 28, 30, 31, 33, 35, 36, 37, 39, 43, 44, 50, 51, 54, 57, 58]

trainloader, testloader  = load_wider(train_file, test_file, class_labels, batch_size, num_workers)



Compose(
    RandAugment(num_ops=2, magnitude=9, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    Resize(size=(226, 226), interpolation=bilinear, max_size=None, antialias=warn)
)
Compose(
    Resize(size=(226, 226), interpolation=bilinear, max_size=None, antialias=warn)
)


In [6]:
from collections import Counter

# Iterate through batches in the trainloader
for batch_idx, batch in enumerate(trainloader):
    # Only process the first 3 batches
    if batch_idx >= 3:
        break

    # Extract labels for the current batch
    batch_labels = batch['label']

    # Convert batch_labels to a list if it's not already (e.g., if it's a tensor)
    if not isinstance(batch_labels, list):
        batch_labels = batch_labels.tolist()

    # Count the frequency of each label in this batch
    label_counts = Counter(batch_labels)

    # Print the label counts for this batch
    print(f"Batch {batch_idx + 1} class counts:")
    for label in range(30):  # Assuming classes are labeled from 0 to 29
        print(f"  Class {label}: {label_counts[label]} instances")
    print("-" * 30)  # Just a separator for clarity

Batch 1 class counts:
  Class 0: 12 instances
  Class 1: 2 instances
  Class 2: 5 instances
  Class 3: 5 instances
  Class 4: 5 instances
  Class 5: 10 instances
  Class 6: 4 instances
  Class 7: 3 instances
  Class 8: 7 instances
  Class 9: 10 instances
  Class 10: 4 instances
  Class 11: 16 instances
  Class 12: 0 instances
  Class 13: 8 instances
  Class 14: 1 instances
  Class 15: 11 instances
  Class 16: 3 instances
  Class 17: 8 instances
  Class 18: 2 instances
  Class 19: 29 instances
  Class 20: 6 instances
  Class 21: 13 instances
  Class 22: 17 instances
  Class 23: 11 instances
  Class 24: 9 instances
  Class 25: 10 instances
  Class 26: 14 instances
  Class 27: 14 instances
  Class 28: 10 instances
  Class 29: 7 instances
------------------------------
Batch 2 class counts:
  Class 0: 14 instances
  Class 1: 8 instances
  Class 2: 5 instances
  Class 3: 11 instances
  Class 4: 5 instances
  Class 5: 6 instances
  Class 6: 6 instances
  Class 7: 1 instances
  Class 8: 2 ins

# Train Teacher

## Prep s3 bucket

In [7]:
# Calling the function
model_name = 'resnet32x4_wider'
model_weight_path = 'weights/wider_teacher_resnet32x4_weights.pth'
num_class = 30
data_name = 'wider'  
batch_size = 32  
bucket_name = '210bucket'  

## Load in models

### resnet32x4_wider

In [8]:
# Instantiate the models
teacher_name = 'resnet32x4_wider'
teacher_model = models_package.__dict__[teacher_name](num_class=30)
teacher_model.fc = nn.Linear(teacher_model.fc.in_features, 30)

### resnet8x4_wider

In [9]:
student_name = 'resnet8x4_wider'
student_model = models_package.__dict__[student_name](num_class=30)
student_model.fc = nn.Linear(teacher_model.fc.in_features, 30)

In [10]:
# Optimizer and scheduler for the student model
student_optimizer = optim.SGD(student_model.parameters(), lr=learning_rate, momentum=momentum)
student_scheduler = torch.optim.lr_scheduler.StepLR(student_optimizer, step_size=step_size, gamma=gamma)

# Optimizer and scheduler for the teacher model
teacher_optimizer = optim.SGD(teacher_model.parameters(), lr=learning_rate, momentum=momentum)
teacher_scheduler = torch.optim.lr_scheduler.StepLR(teacher_optimizer, step_size=step_size, gamma=gamma)

criterion = nn.CrossEntropyLoss()
# Assuming the device is a CUDA device if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
teacher_lr = best_LR_wider('resnet32x4_lr_wider_test', teacher_model, trainloader, criterion, teacher_optimizer, teacher_scheduler, device, num_epochs=3)
teacher_lr

  0%|                                                                                                 | 0/41 [00:10<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.12 GiB. GPU 0 has a total capacty of 22.19 GiB of which 2.98 GiB is free. Including non-PyTorch memory, this process has 19.21 GiB memory in use. Of the allocated memory 18.88 GiB is allocated by PyTorch, and 31.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF